In [ ]:
!pip install transformers
!pip install nlpaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 29.3 MB/s 
     |████████████████████████████████| 182 kB 56.3 MB/s 
     |████████████████████████████████| 7.6 MB 57.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 410 kB 21.7 MB/s 


In [ ]:
import nlpaug.augmenter.word.context_word_embs as aug

In [1]:
import pandas as pd

df = pd.read_csv('/content/augmented35.csv')
df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Text,Sentiment,probability,Condition,Unnamed: 4,Unnamed: 5
0,2757,4007.0,7936.0,Probably a couple hours tops,7.0,0.982308,Confident,NaN,NaN
1,5010,850.0,3441.0,"weather and energy price data elena , pleas...",2.0,0.995415,Confident,NaN,NaN
2,2837,211.0,3958.0,outsourcing communication access . benefits ...,1.0,0.980313,Confident,NaN,NaN
3,412,7471.0,5975.0,$ 3 leads ! click here to have a representat...,2.0,0.964796,Confident,NaN,NaN
4,7388,7304.0,8599.0,['iso new england and pjm propose standard mar...,6.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
9793,1067,7660.0,5253.0,MILLION POUNDS TO YOURACCOUNT.My name is Mr.Ja...,2.0,0.982475,Confident,NaN,NaN
9794,684,6224.0,5581.0,Dear Cherie--I could make time to meet in DC d...,2.0,0.983043,Confident,NaN,NaN
9795,9034,6301.0,5577.0,Oh god. I'm gonna Google nearby cliffs now.,7.0,0.995223,Confident,NaN,NaN
9796,9542,NaN,NaN,['our valentines registration day special! win...,1.0,NaN,NaN,NaN,NaN


In [2]:
df['Sentiment'].value_counts()

2.0    4147
7.0    1970
5.0     800
6.0     737
1.0     722
3.0     711
4.0     711
Name: Sentiment, dtype: int64

##NLP sentence augmentation

In [ ]:
sample_text = df['Text'].iloc[100]

In [ ]:
sample_text

'FIRM Fogarty Daniel .1Sent: Tuesday April 26 2011 8:46 AMTo Mills Cheryl D; StIlin Jacob i Abedin Hume; Web Ake GCc S_SpeeletAardstantsSubject: Stevens Update On Benghazi -- 4/26 7arnAll:Ops has provided below the latest Stevens update on Benghazi:• Traffkaccidenti Police have accepted the USG version of the story: that it was an accident and thevictims were not wearing seathelts. The USG employees were returning from the airport when theincident occurred. A total of four people died and six injured. USG employee Mjuries are light if any.Our people are standing by pending official USG instructions. Stevens notes there is a UK flighttomorrow they could be on if necessary. The TNC wants to do a press release to quell the rumor mill.Stevens will send the draft back to Department for L and PA clearance.• The British envoy departs tomorrow on permanent change of station. There will be a gap before hisreplacement John Jenkins arrives from Baghdad.• INC head Malunoud fibril will be going to 

In [ ]:
augmenter = aug.ContextualWordEmbsAug(model_path='bert-base-uncased', action="insert")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
augmented_sample_text = augmenter.augment(sample_text)

In [ ]:
augmented_sample_text

['firm fogarty daniel. 1sent : tuesday april 26 2011 8 : 46 amto mills cheryl d ; stilin jacob i abedin hume ; web ake gcc s _ speeletaardstantssubject : stevens update on benghazi - - 4 / 26 7arnall : ops has provided below to the latest stevens update on benghazi : • anonymous traffkaccidenti the police have accepted the usg file version of the story : that it was an accident and thevictims were not wearing seathelts. the usg employees were returning from the airport when theincident occurred. a total of four people died and six injured. usg employee mjuries are save light if any. our people are standing by pending official usg instructions. stevens notes there is a uk flighttomorrow they could be on if necessary. the tnc wants not to do a press release to quell the rumor mill. stevens will send the draft back to department for l and pa clearance. • the british envoy departs tomorrow on permanent change of station. there might will be a gap before hisreplacement john jenkins arrives 

In [ ]:
df['Sentiment'].value_counts()

2    4147
7    1970
5     600
1     322
4     311
6     237
3     211
Name: Sentiment, dtype: int64

In [ ]:
from sklearn.utils import shuffle

In [ ]:
from tqdm.auto import tqdm

In [ ]:
import numpy as np

##Function that is generating new samples 

In [ ]:
def augmentMyData(df, augmenter, repetitions=1, samples=200):
    augmented_texts = []
    # select only the minority class samples
    spam_df = df[df['Sentiment'] == 1.0].reset_index(drop=True) # removes unecessary index column
    for i in tqdm(np.random.randint(0, len(spam_df), samples)):
        # generating 'n_samples' augmented texts
        for _ in range(repetitions):
            augmented_text = augmenter.augment(spam_df['Text'].iloc[i])
            augmented_texts.append(augmented_text)
    
    data = {
        'Sentiment': 1.0,
        'Text': augmented_texts
    }
    aug_df = pd.DataFrame(data)
    df = shuffle(df.append(aug_df).reset_index(drop=True))
    return df

In [ ]:
aug_df_faci = augmentMyData(df, augmenter, samples=400)

  0%|          | 0/400 [00:00<?, ?it/s]

In [ ]:
aug_df_faci['Sentiment'].value_counts()

2.0    4147
7.0    1970
5.0     800
6.0     737
1.0     722
3.0     711
4.0     711
Name: Sentiment, dtype: int64

In [ ]:
def augmentMyData(df, augmenter, repetitions=1, samples=200):
    augmented_texts = []
    # select only the minority class samples
    spam_df = df[df['Sentiment'] == 4.0].reset_index(drop=True) # removes unecessary index column
    for i in tqdm(np.random.randint(0, len(spam_df), samples)):
        # generating 'n_samples' augmented texts
        for _ in range(repetitions):
            augmented_text = augmenter.augment(spam_df['Text'].iloc[i])
            augmented_texts.append(augmented_text)
    
    data = {
        'Sentiment': 4.0,
        'Text': augmented_texts
    }
    aug_df = pd.DataFrame(data)
    df = shuffle(df.append(aug_df).reset_index(drop=True))
    return df

In [ ]:
aug_df_with_both = augmentMyData(aug_df_faci, augmenter, samples=400)

  0%|          | 0/400 [00:00<?, ?it/s]

In [ ]:
aug_df_with_both['Sentiment'].value_counts()

2.0    4147
7.0    1970
5.0     800
6.0     737
4.0     711
3.0     711
1.0     322
Name: Sentiment, dtype: int64

In [ ]:
aug_df_faci.to_csv('augmented35.csv')

In [ ]:
# df_class_1 = aug_df_with_both[aug_df_with_both['sentiments'] == 1]
df_class_2 = aug_df_with_both[aug_df_with_both['sentiments'] == 2]
df_class_3 = aug_df_with_both[aug_df_with_both['sentiments'] == 3]

In [ ]:
len(df_class_2)

656

In [ ]:
aug_df_with_both

,Clickbaits,sentiments
1776,"যদি পরিয না গরাহক, আইএনআর [UNK], [UNK]. [UNK] ...",3.0
2120,বহিরগামী কল / ডেটা পরিষেবাগলি এযারটেল নং ২ [UN...,3.0
2102,তার অলিভা তার একটি দলিল পত্রিকা পাঠিযেছে,3.0
2010,পরিবারের লোকেরা একই বর্ণ নামের লোকদের সাথে এ...,3.0
612,তাহলে আপনি আপনার ঠিক একটি বিষয় ভাল বযবসাযিক প...,2.0
...,...,...
2162,বাজফিড অবলম্বনে পরেস । বলগ,3.0
1575,আমরা জানি আপনি ভালোবাসা দিবসে একা থাকবেন কিনা,1.0
1268,আইসক্রিম হ'ল শয়তান কীভাবে বিশ্বকে ধ্বংস করতে ...,1.0
916,"নাটালি ডর্মার ""গেম অফ থ্রোনস"" তে সম্পূর্ণ ভিন্...",1.0


In [ ]:
aug_df_with_both['sentiments'].value_counts()

1.0    897
2.0    656
3.0    632
Name: sentiments, dtype: int64

In [ ]:
df_class_0_under = df_class_1.sample(900)

In [ ]:
df_test_under = pd.concat([df_class_0_under, df_class_2,df_class_3], axis=0)

In [ ]:
df_test_under['sentiments'].value_counts()

NameError: ignored

In [ ]:
df_test_under.to_excel('Balenced_data_bengali.xlsx')